# VERSÃO DO AGENTE QUE USA SOMENTE TABELA FATO_VENDA

### Criação do modelo do agente:

In [0]:
%skip

# =========================================================
# DATAEXPERTS — CRIAÇÃO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow
from mlflow.pyfunc import PythonModel
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from mlflow.deployments import get_deploy_client

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
EXPERIMENT_ID = "4074425516892918"
MODEL_NAME = "dataex_sql_agent"
LLM_ENDPOINT = "databricks-llama-4-maverick"

CATALOG = "dataexperts"
SCHEMA = "gold"
FACT_TABLE = "fato_vendas"

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# ---------------------------------------------------------
# ASSINATURA DO MODELO
# ---------------------------------------------------------
input_schema = Schema([
    ColSpec("string", "question")
])

output_schema = Schema([
    ColSpec("string", "sql"),
    ColSpec("string", "answer")
])

signature = ModelSignature(inputs=input_schema, outputs=output_schema)

input_example = {
    "question": "Qual foi o faturamento total por mês em 2024?"
}

# ---------------------------------------------------------
# CLIENTE LLM
# ---------------------------------------------------------
client = get_deploy_client("databricks")

# ---------------------------------------------------------
# AGENTE NL → SQL
# ---------------------------------------------------------
class DataExSQLAgent(PythonModel):

    @mlflow.trace
    def predict(self, context, model_input):
        question = model_input["question"]

        system_prompt = f"""
Você é um agente especialista em SQL no Databricks, focado em consultas ad-hoc de análise de dados.
Tabela disponível: {CATALOG}.{SCHEMA}.{FACT_TABLE}
Colunas existentes:
- sk_data
- venda_id
- categoria_id
- quantidade
- preco_lista
- valor_total
- _source_file
- _ingestion_date
- sk_cliente
- sk_produto
- sk_localidade
- ano
- mes

Regras obrigatórias:
- Responda APENAS com a consulta SQL correspondente, sem explicações ou texto adicional.
- Use SOMENTE a tabela acima (não utilize outras tabelas).
- Para faturamento, use SUM(valor_total) como total de vendas.
- Para quantidade total, use SUM(quantidade).
- Para contagem de vendas, use COUNT(venda_id).
- Para preço médio, use AVG(preco_lista).
- Para filtros de data, use as colunas ano e mes (ex: ano = 2024 e mes = 6).
- Comece a consulta sempre com SELECT (comando SQL).
- Use aliases claros (por ex. `f` para fato_vendas).
- Nunca crie colunas ou tabelas que não existem no esquema.
- Não use Markdown nem comentários, retorne apenas o SQL final.
"""


        user_prompt = f"""
Pergunta:
{question}

Retorne apenas o SQL.
"""

        response = client.predict(
            endpoint=LLM_ENDPOINT,
            inputs={
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.0
            }
        )

        sql = response["choices"][0]["message"]["content"].strip()

        if context is None:
            return {"sql": sql, "answer": ""}

        df = spark.sql(sql)

        return {
            "sql": sql,
            "answer": df.toPandas().to_json(orient="records")
        }

# ---------------------------------------------------------
# LOG DO MODELO
# ---------------------------------------------------------
with mlflow.start_run(run_name="dataex-sql-agent-fato-only"):
    mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model=DataExSQLAgent(),
        registered_model_name=MODEL_NAME,
        signature=signature,
        input_example=input_example
    )

### Execução do modelo do agente com a pergunta personalizada:

In [0]:
# =========================================================
# DATAEXPERTS — USO DO NL → SQL AGENT (SOMENTE FATO)
# =========================================================

import mlflow

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
MODEL_NAME = "dataex_sql_agent"

# Unity Catalog → SEMPRE usar alias
MODEL_URI = f"models:/{MODEL_NAME}@somente_tabela_fato_venda"

# ---------------------------------------------------------
# CARREGAR O AGENT
# ---------------------------------------------------------
agent = mlflow.pyfunc.load_model(MODEL_URI)

print(f"✅ Agent carregado: {MODEL_URI}")

# ---------------------------------------------------------
# PERGUNTA
# ---------------------------------------------------------
pergunta = "Quais são todas as categorias em ordem sem repetir?"

resultado = agent.predict({
    "question": pergunta
})

# ---------------------------------------------------------
# RESULTADOS
# ---------------------------------------------------------
print("SQL GERADO:")
print(resultado["sql"])

print("\nRESPOSTA:")
print(resultado["answer"])


### Mostrando resultado da pergunta personalizada:

In [0]:
# =========================================================
# DATAEXPERTS — EXECUÇÃO AUTOMÁTICA DO SQL GERADO PELO AGENT
# =========================================================

sql_gerado = resultado["sql"]

print("🚀 Executando SQL gerado pelo agente:")
print(sql_gerado)

# Executa no Spark
df_resultado = spark.sql(sql_gerado)

# Exibe como tabela no Databricks
display(df_resultado)


In [0]:

# Apenas para testes: 

tb = spark.read.format("delta").load("/Volumes/dataexperts/gold/main/fato_vendas")

display(tb)